MLPClassifier example 

In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns

## Load in the file Dataset_spine.csv

- Use Pandas
- Drop the column Unnamed: 13

Always inspect the dataframe using .head()

In [ ]:
# your code here



Also drop ['Col7','Col8','Col9','Col10','Col11','Col12']

In [ ]:
# your code here



This will show the classes, obviously that column will be our y, the rest will be X

In [ ]:
df["Class_att"].unique()

Binary Classifier

import the things we need from sklearn.neural_network import MLPClassifier

train_test_split
confusion_matrix

In [ ]:
# your code here 



Set the variables X and y by picking the columns as discussed above

Do a train_test_split, use a fixed random_state and make the test_size be 25%

Now lets use MLPClassifier from sklearn

Do for 3 hidden layers with 100 units in each

set tol=0.000000001

use solver 'sgd'

In [ ]:
MLPClassifier?

fit the classifier using training

Get y_pred using .predict

Get the accuracy score

Get the Confusion Matrix

Let's view a heatmap with the confusion matrix

In [ ]:
sns.heatmap(cm, center=True)
plt.show()

# Tensorflow

Now what about Tensorflow??

In [ ]:
import tensorflow as tf

Set up a new sequential model, call it model_tf

Have 3 hidden layers, each with activation relu

have an output layer with only 1 unit, no activation function

Should you also set up a normaliser as well. See example workbooks

Compile the model, pick an optimizer, use Adam or tf.keras.optimizers.experimental.SGD(0.001). loss is binarycrossentropy

    metrics=['accuracy']

In [ ]:
y_train

Now try fitting, 20 epochs, do 

    X_train.values, y_train.values, epochs=20, validation_split=0.2

 You should get a bunch of errors

### Why this error?

Well look at y_train

In [ ]:
y_train

They're not numbers, we need to convert those to numbers. Using LabelEncoder

Scikit-learn didn't care, but Tensorflow is a "lower-level" programming tool so you need to do the conversion yourself

In [ ]:
from sklearn import preprocessing

In [ ]:
# your code here



Fit the label encoder on y_train

Now transform both y_train and y_test

In [ ]:
y_train_enc = 
y_test_enc = 

In [ ]:
y_train_enc

## Fitting again

Try fitting your model again with y_train_enc. It should work now

In [ ]:
# your code here



Accuracy for both the training and validation sets is a little all over the place. Maybe we need more Epochs or some other way to decide when to finish. Anyway...

In [ ]:
y_pred = model_tf.predict(X_test)

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
y_pred

Those are not probabilities. Damn. Well we need to do some more

We could have set the activation to sigmoid on the last layer and that would've given us probabilities but tensorflow manual says not to do that

In [ ]:
y_probs = tf.keras.activations.sigmoid(y_pred)

Let's convert those probabilities to 0s or 1s. This is not the best way of doing this, Tensorflow almost certainly has something better, but this is how I want to do it!

In [ ]:
results = pd.DataFrame(y_probs, columns=["value"])

In [ ]:
results["which"] = 0

In [ ]:
results.loc[results['value'] >= 0.5, 'which'] = 1

In [ ]:
results

In [ ]:
results.which.values

In [ ]:
y_test_enc

In [ ]:
mistakes = (results.which.values != y_test_enc).sum()

In [ ]:
model_tf.evaluate(X_test, y_test_enc)

In [ ]:
1-mistakes/78

Ok, it's worse than the sklearn implementation in test data but I haven't tried anything to tweak it, sklearn you will have noticed did more epochs in its training. It also didn't use any validation set.

## More Epochs! Lower learning rate

I'm going to take the same basic structure and see what happens when I make some changes

In [ ]:
model_tf = tf.keras.Sequential([
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(1)
])

I'll also do a lower learning rate

In [ ]:
model_tf.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

Lots and lots of epochs, let's run it and store the history

In [ ]:
history = model_tf.fit(X_train.values, y_train_enc, epochs=1000, validation_split=0.2)

In [ ]:
model_tf.summary()

In [ ]:
model_tf.evaluate(X_test, y_test_enc)

In [ ]:
def plot_loss(history, which='loss'):
    plt.plot(history.history[which], label='train')
    try:
        plt.plot(history.history['val_'+which], label='validation')
    except:
        None
    plt.xlabel('Epoch')
    plt.ylabel(which)
    plt.legend()
    plt.grid(True)

In [ ]:
plot_loss(history)

In [ ]:
plot_loss(history, 'accuracy')

So probably shouldn't go for 1000 epochs, there's overfitting there

I'm going to implement Early Stopping. It's a form of regularisation - to try to prevent overfitting, and it also will train our model more quickly.

<a href="https://en.wikipedia.org/wiki/Early_stopping">Wikipedia Link</a>

<a href="https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/">More Early Stopping Detail</a>

<a href="https://towardsdatascience.com/early-stopping-a-cool-strategy-to-regularize-neural-networks-bfdeca6d722e">Another Link</a>

I mentioned it in lectures without going into any detail and will talk about it more next week.

The basic idea is, monitor how well the validation set is performing after every epoch, if there has not been any improvement in its score for a while (say in 10 epochs or whatever you pick as your "patience" - we don't just say over 1 epoch as training could just be "stuck"). MLPClassifier in sklearn has it sort of implemented and we saw it  above "Training loss did not improve more than tol=0.000000 for 10 consecutive epochs. Stopping."

It is added as a callback function that checks for what we want

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

Be careful with restore_best_weights see https://medium.com/@doleron/never-use-restore-best-weights-true-with-earlystopping-754ba5f9b0c6

In [ ]:
#callback_modelcheckpoint_loss = tf.keras.callbacks.ModelCheckpoint(filepath='best_model_loss.keras', monitor='val_loss', save_best_only=True)

In [ ]:
model_tf2 = tf.keras.Sequential([
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(1)
])

In [ ]:
model_tf2.compile(
     optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [ ]:
model_tf2.fit(X_train.values, y_train_enc, epochs=1000, validation_split=0.2, callbacks=[callback])

I said go for 1000 epochs, but will only go for a certain amount due to early stopping

I also said to restore_best_weights, so it should be the model with the lowest val_loss that was found

In [ ]:
model_tf2.evaluate(X_test, y_test_enc)

This may be worse than sklearn still

Anything you run may be different due to randomisation of batches etc

You can try adding regularisation. Using a different loss/optimizer and other things to improve your model. Also there is going to be some random chance.